# Day8_1: 시계열 분석 기법 (resample, rolling, 분해) - 정답

## 📚 학습 목표

**Part 1: 기초**
1. resample()로 시계열 데이터 리샘플링하기
2. 업샘플링과 다운샘플링의 차이 이해하기
3. 다양한 집계 함수로 주기 변환하기
4. rolling()으로 이동 평균 계산하기
5. expanding()으로 누적 통계 구하기

**Part 2: 심화**
1. 지수 가중 이동 평균(EWM) 활용하기
2. 시계열 분해로 추세와 계절성 추출하기
3. Plotly로 시계열 분해 결과 시각화하기
4. 이동 통계를 활용한 이상치 탐지하기

---

In [ ]:
# 필요한 라이브러리 임포트
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
from statsmodels.tsa.seasonal import seasonal_decompose

print("✅ 라이브러리 로드 완료")

---

## 🎯 실습 퀴즈 정답

---

### Q1. 일별 매출을 주별로 집계하기 ⭐

**문제**: 다음 일별 매출 데이터를 주별로 다운샘플링하여 합계를 구하세요.

In [ ]:
# Q1 정답

dates = pd.date_range('2024-01-01', periods=14, freq='D')
sales = pd.DataFrame({'revenue': [100, 120, 90, 110, 130, 140, 95, 105, 125, 115, 135, 145, 100, 120]}, index=dates)

# 주별 합계 계산
weekly_sales = sales.resample('W').sum()

print("주별 매출 합계:")
print(weekly_sales)

In [ ]:
# 테스트
assert len(weekly_sales) == 3, "주별 데이터는 3개여야 합니다"
assert weekly_sales['revenue'].sum() == sales['revenue'].sum(), "총 합계가 일치해야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.resample('W')`로 주별(Weekly) 주기로 다운샘플링
- `.sum()`으로 각 주의 합계 계산

**핵심 개념**:
- `resample()`: DatetimeIndex가 있는 시계열 데이터의 주기 변경
- `'W'`: 주별 주기 (일요일 기준)
- 다운샘플링: 고빈도 → 저빈도 (일별 → 주별)

**대안 솔루션**:
```python
# 특정 요일 기준으로 집계 (예: 월요일)
weekly_sales_mon = sales.resample('W-MON').sum()
```

**흔한 실수**:
- ❌ `.groupby(pd.Grouper(freq='W'))`로 복잡하게 작성
- ❌ DatetimeIndex 없이 resample 시도 → 에러

**실무 팁**:
일별 데이터를 주별/월별로 요약할 때 resample은 필수! 매출 보고서, KPI 대시보드에서 자주 사용됩니다.

---

### Q2. 7일 이동 평균 계산하기 ⭐⭐

**문제**: 주어진 매출 데이터에 7일 이동 평균 컬럼을 추가하세요.

In [ ]:
# Q2 정답

dates = pd.date_range('2024-01-01', periods=30, freq='D')
sales = pd.DataFrame({'revenue': np.random.randint(80, 150, size=30)}, index=dates)

# 7일 이동 평균 계산
sales['MA_7'] = sales['revenue'].rolling(window=7).mean()

print("7일 이동 평균:")
print(sales.head(15))

In [ ]:
# 테스트
assert 'MA_7' in sales.columns, "MA_7 컬럼이 있어야 합니다"
assert pd.isna(sales.iloc[0]['MA_7']), "처음 6개는 NaN이어야 합니다"
assert not pd.isna(sales.iloc[6]['MA_7']), "7번째부터 값이 있어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.rolling(window=7)`로 7일 윈도우 설정
- `.mean()`으로 각 윈도우의 평균 계산

**핵심 개념**:
- `rolling()`: 이동 윈도우를 만들어 통계 계산
- 처음 6개는 NaN (윈도우 크기 미달)
- 7번째부터 이동평균 계산 시작

**대안 솔루션**:
```python
# 최소 데이터 개수 지정 (결측치 줄이기)
sales['MA_7_min3'] = sales['revenue'].rolling(window=7, min_periods=3).mean()
```

**흔한 실수**:
- ❌ `window=7`을 빼먹고 `.rolling().mean()` 호출
- ❌ 첫 6개 NaN을 보고 에러로 착각

**실무 팁**:
주가 차트의 5일선, 20일선이 바로 이동평균입니다. 노이즈 제거하고 추세 파악에 필수!

---

### Q3. 누적 합계 구하기 ⭐⭐

**문제**: 일별 수익에 대해 연초부터 현재까지 누적 합계를 계산하세요.

In [ ]:
# Q3 정답

dates = pd.date_range('2024-01-01', periods=10, freq='D')
profit = pd.DataFrame({'profit': [50, -20, 30, 40, -10, 60, 20, 30, -15, 45]}, index=dates)

# 누적 합계 계산
profit['cumulative_profit'] = profit['profit'].expanding().sum()

print("누적 수익:")
print(profit)

In [ ]:
# 테스트
assert 'cumulative_profit' in profit.columns, "cumulative_profit 컬럼이 있어야 합니다"
assert profit.iloc[0]['cumulative_profit'] == 50, "첫날 누적은 50이어야 합니다"
assert profit.iloc[-1]['cumulative_profit'] == profit['profit'].sum(), "마지막 누적은 전체 합계와 같아야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.expanding()`으로 처음부터 현재까지 윈도우 확장
- `.sum()`으로 누적 합계 계산

**핵심 개념**:
- `expanding()`: 윈도우가 계속 확장되는 누적 통계
- 음수 수익도 누적됨 (수익 - 손실)
- 연초부터 현재까지 성과 추적에 활용

**대안 솔루션**:
```python
# pandas cumsum() 메서드 사용
profit['cumulative_profit'] = profit['profit'].cumsum()
```

**흔한 실수**:
- ❌ `rolling()`과 혼동하여 window 파라미터 지정
- ❌ 누적 평균이 필요한데 누적 합계를 계산

**실무 팁**:
연초부터 현재까지 누적 매출, 누적 비용 등 YTD (Year-to-Date) 지표 계산에 필수!

---

### Q4. 월별 OHLC 집계 ⭐⭐⭐

**문제**: 일별 주가 데이터를 월별로 다운샘플링하여 OHLC (Open, High, Low, Close)를 계산하세요.

In [ ]:
# Q4 정답

dates = pd.date_range('2024-01-01', '2024-03-31', freq='D')
stock = pd.DataFrame({'close': 50000 + np.cumsum(np.random.randn(len(dates)) * 500)}, index=dates)

# 월별 OHLC 계산
monthly_ohlc = stock.resample('M').agg({
    'close': ['first', 'max', 'min', 'last']
})

# 컬럼명 변경
monthly_ohlc.columns = ['Open', 'High', 'Low', 'Close']

print("월별 OHLC:")
print(monthly_ohlc)

In [ ]:
# 테스트
assert len(monthly_ohlc) == 3, "3개월 데이터여야 합니다"
assert list(monthly_ohlc.columns) == ['Open', 'High', 'Low', 'Close'], "OHLC 컬럼이 있어야 합니다"
assert (monthly_ohlc['High'] >= monthly_ohlc['Low']).all(), "High는 Low보다 커야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.resample('M')`로 월별 다운샘플링
- `.agg()`로 여러 집계 함수 동시 적용
- `first` (시가), `max` (고가), `min` (저가), `last` (종가)

**핵심 개념**:
- `agg()`: 여러 집계 함수를 딕셔너리로 지정
- OHLC: 주식 차트의 기본 데이터 (캔들스틱 차트)
- `'M'`: 월말(Month End) 기준

**대안 솔루션**:
```python
# ohlc() 메서드 사용 (OHLC 전용)
monthly_ohlc = stock.resample('M')['close'].ohlc()
```

**흔한 실수**:
- ❌ `first`, `last` 대신 `mean`, `median` 사용
- ❌ 컬럼명을 변경하지 않아 ('close', 'first') 같은 MultiIndex 발생

**실무 팁**:
주가 분석, 암호화폐 차트에서 OHLC는 필수! Plotly의 `go.Candlestick()`으로 시각화 가능합니다.

---

### Q5. 업샘플링 후 선형 보간 ⭐⭐⭐

**문제**: 월별 예산 데이터를 일별로 업샘플링한 후 선형 보간(interpolate)하세요.

In [ ]:
# Q5 정답

monthly_budget = pd.DataFrame({'budget': [1000, 1200, 1100]}, index=pd.date_range('2024-01', periods=3, freq='MS'))

# 일별 업샘플링 후 선형 보간
daily_budget = monthly_budget.resample('D').interpolate(method='linear')

print("일별 예산 (처음 35일):")
print(daily_budget.head(35))

In [ ]:
# 테스트
assert len(daily_budget) > len(monthly_budget), "일별 데이터가 더 많아야 합니다"
assert not daily_budget['budget'].isna().any(), "NaN이 없어야 합니다"
assert daily_budget.iloc[0]['budget'] == 1000, "첫 값은 1000이어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.resample('D')`로 일별 업샘플링
- `.interpolate(method='linear')`로 선형 보간

**핵심 개념**:
- 업샘플링: 저빈도 → 고빈도 (월별 → 일별)
- 선형 보간: 두 값 사이를 직선으로 연결하여 중간값 계산
- 결측치(NaN) 자동 채우기

**대안 솔루션**:
```python
# Forward Fill (계단식)
daily_budget_ffill = monthly_budget.resample('D').ffill()

# Backward Fill (뒤에서 채우기)
daily_budget_bfill = monthly_budget.resample('D').bfill()
```

**흔한 실수**:
- ❌ `.asfreq()`만 쓰고 보간 안 함 → NaN 가득
- ❌ `method='linear'` 빼먹음

**실무 팁**:
월별 예산을 일별로 분배하거나, 주간 데이터를 일별로 확장할 때 유용! 마케팅 예산 배분, 생산 계획 수립에 활용됩니다.

---

### Q6. 이동 표준편차로 변동성 측정 ⭐⭐⭐

**문제**: 주가 데이터에서 20일 이동 표준편차를 계산하여 변동성을 측정하세요.

In [ ]:
# Q6 정답

dates = pd.date_range('2024-01-01', periods=60, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(60) * 2)}, index=dates)

# 20일 이동 표준편차 (변동성)
stock['volatility'] = stock['price'].rolling(window=20).std()

print("변동성 측정:")
print(stock[['price', 'volatility']].tail(20))

In [ ]:
# 테스트
assert 'volatility' in stock.columns, "volatility 컬럼이 있어야 합니다"
assert pd.isna(stock.iloc[18]['volatility']), "19번째까지 NaN이어야 합니다"
assert not pd.isna(stock.iloc[19]['volatility']), "20번째부터 값이 있어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.rolling(window=20)`로 20일 윈도우 설정
- `.std()`로 표준편차 계산

**핵심 개념**:
- 변동성(Volatility): 가격의 변동 정도 (표준편차로 측정)
- 높을수록 리스크 증가, 낮을수록 안정적
- 금융 시장 분석의 핵심 지표

**대안 솔루션**:
```python
# 일별 수익률의 변동성
stock['daily_return'] = stock['price'].pct_change()
stock['return_volatility'] = stock['daily_return'].rolling(20).std()
```

**흔한 실수**:
- ❌ `.mean()` 사용 → 변동성이 아닌 평균
- ❌ 윈도우 크기 너무 작음 → 노이즈 과다

**실무 팁**:
변동성이 높아지면 옵션 가격 상승, 리스크 관리 강화! 주식 포트폴리오 관리, VaR 계산에 필수입니다.

---

### Q7. 지수 가중 이동 평균 (EWM) 계산 ⭐⭐⭐⭐

**문제**: 주가 데이터에서 span=10인 지수 가중 이동 평균을 계산하세요.

In [ ]:
# Q7 정답

dates = pd.date_range('2024-01-01', periods=50, freq='D')
stock = pd.DataFrame({'price': 100 + np.cumsum(np.random.randn(50) * 2)}, index=dates)

# 지수 가중 이동 평균 계산
stock['EWM_10'] = stock['price'].ewm(span=10, adjust=False).mean()

print("EWM 계산 결과:")
print(stock[['price', 'EWM_10']].head(20))

In [ ]:
# 테스트
assert 'EWM_10' in stock.columns, "EWM_10 컬럼이 있어야 합니다"
assert not stock['EWM_10'].isna().any(), "EWM은 첫 값부터 계산됩니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `.ewm(span=10, adjust=False)`로 EWM 객체 생성
- `.mean()`으로 지수 가중 평균 계산

**핵심 개념**:
- EWM: 최근 데이터에 더 큰 가중치 부여
- `span`: 윈도우 크기 (클수록 평활화)
- `adjust=False`: 재귀적 계산 (금융에서 표준)
- SMA보다 최근 변화에 민감

**대안 솔루션**:
```python
# alpha 직접 지정 (0~1 사이)
stock['EWM_alpha'] = stock['price'].ewm(alpha=0.2, adjust=False).mean()
```

**흔한 실수**:
- ❌ `adjust=True` 사용 → 다른 결과
- ❌ `span`과 `alpha` 동시 지정 → 에러

**실무 팁**:
MACD, RSI 같은 기술 지표가 모두 EWM 기반! 트레이딩 시스템, 이상 탐지에서 널리 사용됩니다.

---

### Q8. 시계열 분해 수행하기 ⭐⭐⭐⭐

**문제**: 다음 월별 매출 데이터를 Additive 모델로 분해하여 추세, 계절성, 잔차를 추출하세요.

In [ ]:
# Q8 정답

dates = pd.date_range('2022-01', periods=24, freq='MS')
sales = pd.DataFrame({
    'revenue': 100 + np.linspace(0, 50, 24) + 20 * np.sin(np.linspace(0, 4*np.pi, 24)) + np.random.randn(24) * 5
}, index=dates)

# 시계열 분해
decomposition = seasonal_decompose(
    sales['revenue'],
    model='additive',
    period=12  # 12개월 주기
)

# 컴포넌트 추출
trend = decomposition.trend
seasonal = decomposition.seasonal
residual = decomposition.resid

print("추세 (Trend):")
print(trend.head(15))
print("\n계절성 (Seasonality):")
print(seasonal.head(15))
print("\n잔차 (Residual):")
print(residual.head(15))

In [ ]:
# 테스트
assert trend is not None, "추세가 있어야 합니다"
assert seasonal is not None, "계절성이 있어야 합니다"
assert residual is not None, "잔차가 있어야 합니다"
# 가산 모델 검증: 원본 = 추세 + 계절성 + 잔차
reconstructed = trend + seasonal + residual
assert np.allclose(sales['revenue'], reconstructed, equal_nan=True), "분해 결과가 원본과 일치해야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- `seasonal_decompose()` 함수 사용
- `model='additive'`: 가산 모델 (Y = Trend + Seasonality + Residual)
- `period=12`: 12개월 주기 (연간 계절성)

**핵심 개념**:
- **Trend**: 장기적인 상승/하락 패턴
- **Seasonality**: 주기적으로 반복되는 패턴
- **Residual**: 설명되지 않는 랜덤 노이즈
- Additive: 변동폭이 일정할 때 사용

**대안 솔루션**:
```python
# Multiplicative 모델 (변동폭이 증가/감소할 때)
decomposition_mult = seasonal_decompose(
    sales['revenue'],
    model='multiplicative',
    period=12
)
```

**흔한 실수**:
- ❌ `period` 설정 안 함 → 자동 추정 오류
- ❌ 데이터가 period의 2배 미만 → 에러
- ❌ 계절성 없는 데이터에 분해 시도 → 의미 없는 결과

**실무 팁**:
소매업 월별 매출, 전력 소비량, 관광 수요 등 계절성이 강한 데이터 분석 필수! 예측 모델링 전 전처리 단계입니다.

---

### Q9. 계절성 제거 (Deseasonalize) ⭐⭐⭐⭐

**문제**: Q8의 분해 결과를 사용하여 계절성을 제거한 데이터를 생성하세요.

In [ ]:
# Q9 정답

# Q8에서 이미 분해 완료
# 계절성 제거: 원본 - 계절성
sales['deseasonalized'] = sales['revenue'] - seasonal

print("계절성 제거 데이터:")
print(sales[['revenue', 'deseasonalized']].head(15))

In [ ]:
# 테스트
assert 'deseasonalized' in sales.columns, "deseasonalized 컬럼이 있어야 합니다"
# 계절성 제거 = 추세 + 잔차
expected = trend + residual
assert np.allclose(sales['deseasonalized'], expected, equal_nan=True), "계절성이 제거되어야 합니다"
print("✅ 테스트 통과!")

### 💡 풀이 설명

**접근 방법**:
- 원본 데이터에서 계절성 컴포넌트를 빼기
- `원본 - seasonality = trend + residual`

**핵심 개념**:
- Deseasonalize: 계절 효과를 제거한 순수 추세
- 연말 쇼핑 시즌, 명절 효과 등을 배제
- 실제 성장률을 파악하기 위한 필수 단계

**대안 솔루션**:
```python
# Multiplicative 모델의 경우
sales['deseasonalized_mult'] = sales['revenue'] / seasonal
```

**흔한 실수**:
- ❌ Additive 모델인데 나누기 연산
- ❌ Multiplicative 모델인데 빼기 연산

**실무 팁**:
"전년 동기 대비 성장률"을 정확히 파악하려면 계절성 제거 필수! 소매, 관광, 농업 등 계절 영향이 큰 산업에서 필수입니다.

---

### Q10. Bollinger Bands로 이상치 탐지 ⭐⭐⭐⭐⭐

**문제**: 센서 온도 데이터에서 Bollinger Bands를 계산하고, 밴드를 벗어나는 이상치를 탐지하세요.

In [ ]:
# Q10 정답

np.random.seed(42)
dates = pd.date_range('2024-01-01', periods=100, freq='H')
sensor = pd.DataFrame({'temp': 25 + np.random.randn(100) * 2}, index=dates)
sensor.loc[sensor.index[20], 'temp'] = 35  # 이상치
sensor.loc[sensor.index[80], 'temp'] = 15  # 이상치

# 1. 20기간 이동평균
sensor['MA'] = sensor['temp'].rolling(window=20).mean()

# 2. 20기간 이동 표준편차
sensor['STD'] = sensor['temp'].rolling(window=20).std()

# 3. Upper/Lower Bands
sensor['Upper'] = sensor['MA'] + 2 * sensor['STD']
sensor['Lower'] = sensor['MA'] - 2 * sensor['STD']

# 4. 이상치 플래그
sensor['anomaly'] = (
    (sensor['temp'] > sensor['Upper']) |
    (sensor['temp'] < sensor['Lower'])
)

print("이상치 탐지 결과:")
print(sensor[sensor['anomaly']])

In [ ]:
# 테스트
assert 'MA' in sensor.columns, "이동평균이 있어야 합니다"
assert 'Upper' in sensor.columns, "Upper Band가 있어야 합니다"
assert 'Lower' in sensor.columns, "Lower Band가 있어야 합니다"
assert 'anomaly' in sensor.columns, "이상치 플래그가 있어야 합니다"
assert sensor['anomaly'].sum() > 0, "이상치가 탐지되어야 합니다"
print("✅ 테스트 통과!")

In [ ]:
# 4. Plotly 시각화
fig = go.Figure()

# 온도 데이터
fig.add_trace(go.Scatter(
    x=sensor.index, y=sensor['temp'],
    mode='lines', name='온도',
    line=dict(color='gray', width=1)
))

# 이동평균
fig.add_trace(go.Scatter(
    x=sensor.index, y=sensor['MA'],
    mode='lines', name='이동평균 (20)',
    line=dict(color='blue', width=2)
))

# Upper Band
fig.add_trace(go.Scatter(
    x=sensor.index, y=sensor['Upper'],
    mode='lines', name='Upper Band',
    line=dict(color='red', width=1, dash='dash')
))

# Lower Band
fig.add_trace(go.Scatter(
    x=sensor.index, y=sensor['Lower'],
    mode='lines', name='Lower Band',
    line=dict(color='red', width=1, dash='dash')
))

# 이상치 마커
anomalies = sensor[sensor['anomaly']]
fig.add_trace(go.Scatter(
    x=anomalies.index, y=anomalies['temp'],
    mode='markers', name='이상치',
    marker=dict(color='orange', size=12, symbol='x')
))

fig.update_layout(
    title='Bollinger Bands를 활용한 이상치 탐지',
    xaxis_title='시간',
    yaxis_title='온도 (°C)',
    hovermode='x unified'
)

fig.show()

### 💡 풀이 설명

**접근 방법**:
1. 이동평균(MA) 계산 → 중심선
2. 이동 표준편차(STD) 계산 → 변동폭
3. Upper/Lower Band = MA ± 2σ
4. 밴드 벗어나면 이상치
5. Plotly로 시각화

**핵심 개념**:
- **Bollinger Bands**: 주가 기술적 분석 지표
- 2σ 기준 ≈ 95% 신뢰구간
- 밴드 폭이 좁아지면 변동성 감소 (곧 큰 변화 예상)
- 밴드 폭이 넓어지면 변동성 증가

**대안 솔루션**:
```python
# 3σ 기준 (더 엄격한 이상치 기준)
sensor['Upper_3sigma'] = sensor['MA'] + 3 * sensor['STD']
sensor['Lower_3sigma'] = sensor['MA'] - 3 * sensor['STD']

# Z-score 기반
sensor['z_score'] = (sensor['temp'] - sensor['MA']) / sensor['STD']
sensor['anomaly_z'] = sensor['z_score'].abs() > 2
```

**흔한 실수**:
- ❌ Upper/Lower 계산 시 표준편차에 2 곱하기 빼먹음
- ❌ 윈도우 크기 너무 작음 → 과도한 이상치 탐지
- ❌ 시각화 시 이상치 마커 누락

**실무 팁**:
센서 데이터, 주식 가격, 웹 트래픽 이상 탐지에 널리 사용! 실시간 모니터링 시스템에서 자동 알림 트리거로 활용됩니다. 2σ는 False Positive가 많을 수 있으니, 도메인 지식으로 임계값 조정이 중요합니다.

---

## 📊 학습 정리

### Part 1: 기초 핵심 요약

| 개념 | 핵심 메서드 | 실무 활용 |
|------|------------|----------|
| **resample** | `.resample('W').sum()` | 일별 → 주별/월별 집계, 주기 변환 |
| **다운샘플링** | `.resample('M').agg({...})` | 고빈도 → 저빈도 (데이터 요약) |
| **업샘플링** | `.resample('D').ffill()` / `.interpolate()` | 저빈도 → 고빈도 (결측치 채우기) |
| **rolling** | `.rolling(7).mean()` | 이동평균, 노이즈 제거, 추세 파악 |
| **expanding** | `.expanding().sum()` | 누적 합계, 누적 평균, 연초부터 현재까지 |

### Part 2: 심화 핵심 요약

| 기법 | 문법 | 언제 쓰나? |
|------|------|----------|
| **EWM** | `.ewm(span=10).mean()` | 최근 데이터 가중치 높임 (변동성, 트렌드 추적) |
| **시계열 분해** | `seasonal_decompose(model='additive', period=12)` | 추세/계절성/잔차 분리, 예측 모델링 준비 |
| **계절성 제거** | `원본 - seasonal` | 계절 효과 제거 후 순수 성장 추세 파악 |
| **Bollinger Bands** | `MA ± 2*STD` | 이상치 탐지, 변동성 모니터링 |

### 💡 실무 팁

1. **주기 선택**: 데이터 특성에 맞는 주기 사용 (일별 → 주별은 'W', 월별은 'M')
2. **결측치 처리**: 업샘플링 후 `.ffill()` (계단형) vs `.interpolate()` (선형) 선택
3. **이동평균 윈도우**: 짧을수록 민감, 길수록 평활화 (주가: 5일, 20일, 60일선)
4. **EWM vs SMA**: 최근 변화 빠르게 반영하려면 EWM, 안정적이면 SMA
5. **시계열 분해**: 계절성 있는 데이터(월별, 분기별)는 반드시 분해 후 분석
6. **이상치 탐지**: Bollinger Bands의 2σ 기준은 약 95% 신뢰구간
7. **Plotly 시각화**: `hovermode='x unified'`로 여러 시계열 동시 비교
8. **DatetimeIndex 필수**: resample/rolling 사용 전 `.set_index('date')` 필수
9. **OHLC 차트**: Plotly `go.Candlestick()`으로 주가 시각화
10. **변동성 추적**: 금융 리스크 관리, 포트폴리오 최적화에 핵심 지표